**全体の流れ**
1. ライブラリのインポート
2. CPU or GPU の選択
3. データ読み出し(CSV or 標準データセット)
4. データセット関数定義
5. 入力データの訓練/検証/評価用分割
6. データセット作成(入力/検証データの型をテンソルに変換)
7. データローダー作成(訓練/検証データをバッチサイズごとにまとめる)
8. モデル構築
9. インスタンス化、最適化手法の選択、損失関数定義
10. epoch単位で学習
11. テストデータで正解率算出

In [1]:
# モジュールのインポート
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
# 追加モジュール
import torch                         # Tensorの作成や操作
import torch.autograd                # 自動微分機能
import torch.nn as nn                # ニューラルネットワーク
import torch.nn.functional as F      # 関数をメソッドとして提供
import torch.optim as optim          # オプティマイザ
from sklearn.model_selection import train_test_split

import torchvision.datasets          # データセット
import torchvision.models            # AlexNetなど
import torchvision.transforms as transforms        # 画像Augumentation,resize、正規化
import torch.utils.data as tud

In [3]:
# GPUの使用状況確認
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
# データ読み出し (28 x 28 = 784pixel, + label = 785列)
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")              # (42000, 785)
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")                # (28000, 784)

# (ID, label)のデータ列
sample_df = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv") # (28000, 784)

In [5]:
# データセット関数
class DATASET(tud.Dataset):
    def __init__(self, train, test):
        self.X = torch.from_numpy(np.asarray(train)).float()
        self.y= torch.from_numpy(np.asarray(test)).float()

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [6]:
# 前処理-----------------------------------------------------
# ラベルを列から切り離す
X = train_df.drop('label', axis=1)
X = X/255
y = train_df['label']

# 評価用のデータを分ける
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [7]:
# データセット作成
train_dataset = DATASET(X_train, y_train)
val_dataset = DATASET(X_val, y_val)
# データローダー作成
train_dataloader = tud.DataLoader(train_dataset, batch_size=64, shuffle=False, drop_last=False)

In [8]:
# バッチサイズ
batch_size = 256

# DataLoaderでミニバッチサイズ単位の分割、学習時のシャッフルを行う
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size)

### ミニバッチ学習とモデル構築

In [9]:
# モデル構築(全結合層x3)
class Net(nn.Module):
    
    # ネットワーク構造の定義
    def __init__(self, in_size, h_size, out_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(in_size,h_size)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(h_size,out_size)
        self.softmax = torch.nn.Softmax(dim=1)
        
    # 順伝播
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

In [10]:
# インスタンス化(入力:784, 出力:10)
net = Net(784,50,10).to(device)

# GPUへのモデル転送
# net = Net(784,800,400,10).to(device)

# 最適化手法の選択
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)

# 目的関数 (cross_entropyは内部でsoftmaxが使用されている)
# criterion = F.cross_entropy
criterion = nn.CrossEntropyLoss()

#### モデルの学習

In [11]:
# エポックの数
max_epoch = 10

# モデルの初期化
torch.manual_seed(0)

y_axis_list = []
y_acc_list = []

# 学習ループ
for epoch in range(max_epoch):
    
    for batch in train_dataloader:
        
        # バッチサイズ分のサンプルを抽出
        x, t = batch
        
        # パラメータの勾配を初期化
        optimizer.zero_grad()
        
        x = x.to(device)
        t = t.to(torch.long)
        t = t.to(device)
        
        # 予測値の算出
        y = net(x)
        
        # 目標値と予測値から目的関数の値を算出
        loss = criterion(y,t)
                
        # 各パラメータの勾配を算出
        loss.backward()
        
        # パラメータ更新
        optimizer.step()
        
        # ロス値
        y_axis_list.append(loss.detach().cpu().numpy())

    # 計算の型を揃えるため、tensor(float32)に変換
    tmp = torch.tensor(X_val.values.astype(np.float32))
    tmp = tmp.to(device)

    # 予測
    y = net(tmp)

    # 予測
    y_pred = torch.argmax(y,1)

    # numpyに変換
    y_pred = y_pred.to('cpu').detach().numpy().copy()

    # 正解率表示
    from sklearn.metrics import accuracy_score, log_loss # 正解率の評価
    accuracy = accuracy_score(y_pred,y_val) * 100
    
    # 目的関数の値を表示して確認
    print('epoch =', epoch+1, ': loss = ', loss.item(), ': acc = ',accuracy)    

epoch = 1 : loss =  1.6012307405471802 : acc =  84.66666666666667
epoch = 2 : loss =  1.50563383102417 : acc =  93.32142857142857
epoch = 3 : loss =  1.5048248767852783 : acc =  94.57142857142857
epoch = 4 : loss =  1.505307912826538 : acc =  95.32142857142857
epoch = 5 : loss =  1.494537591934204 : acc =  95.79761904761904
epoch = 6 : loss =  1.473000168800354 : acc =  95.80952380952381
epoch = 7 : loss =  1.504635214805603 : acc =  95.48809523809524
epoch = 8 : loss =  1.4791464805603027 : acc =  95.47619047619048
epoch = 9 : loss =  1.4917426109313965 : acc =  95.86904761904762
epoch = 10 : loss =  1.4942227602005005 : acc =  96.3452380952381


In [12]:
# 評価用データの確認------------------------------------------------------
# 計算の型を揃えるため、tensor(float32)に変換
tmp = torch.tensor(X_val.values.astype(np.float32))
tmp = tmp.to(device)

# 予測
y = net(tmp)

# 予測
y_pred = torch.argmax(y,1)

# numpyに変換
y_pred = y_pred.to('cpu').detach().numpy().copy()

# 正解率表示
from sklearn.metrics import accuracy_score, log_loss # 正解率の評価
accuracy = accuracy_score(y_pred,y_val) * 100
print('正解率 = {}'.format(accuracy))

正解率 = 96.3452380952381


In [13]:
# テストデータの確認------------------------------------------------------
tmp = torch.tensor(test_df.values.astype(np.float32))
tmp = tmp.to(device)

# 予測
y = net(tmp)

# 予測
y_pred = torch.argmax(y,1)

# numpyに変換
y_pred = y_pred.to('cpu').detach().numpy().copy()

In [14]:
# submissionファイルを作成する

# x(ImageId)のリスト作成
ImageId = [n+1 for n in range(28000)]

# 出力値(Label)を1次元化する
y_pred = y_pred.flatten()

# データフレーム作成
output = pd.DataFrame(ImageId,columns=['ImageId'])

# Label列を追加する
output['Label'] = y_pred

In [15]:
output.to_csv('/kaggle/working/submission.csv', index=False)